In [1]:
import datetime
import pandas as pd
from urllib.request import urlopen
import numpy as np
import  plotly.graph_objs as go
from datetime import datetime
import itertools
from datetime import timedelta

In [2]:
import ipywidgets as widgets
import math
# Import the library for normalization of the data
import sklearn.preprocessing as preproc

In [3]:
hurdat4 = pd.read_csv('hurdat2_MOD.csv')

In [4]:
hurdat4.head()

,date,storm_id,name,cat,lat,long,max_sust_windspeed(knots),min_pressure(millibars)
0,1851-06-25 00:00:00,AL011851,UNNAMED,HU,28.0,-94.8,80,-999
1,1851-06-25 06:00:00,AL011851,UNNAMED,HU,28.0,-95.4,80,-999
2,1851-06-25 12:00:00,AL011851,UNNAMED,HU,28.0,-96.0,80,-999
3,1851-06-25 18:00:00,AL011851,UNNAMED,HU,28.1,-96.5,80,-999
4,1851-06-25 21:00:00,AL011851,UNNAMED,HU,28.2,-96.8,80,-999


In [5]:
def display():
    print('Input Hurricane Name or \'Exit\' to quit')

In [6]:
#Validation function
def integerValidation(msg = "Input year: "):
    validAnswer = False
    while(not validAnswer):
        try:
            YEAR = int(input(msg2))
            if int(1851) <= YEAR <= int(2021):
                validAnswer = True
        except:
            print("This must be an integer number and between the range 1851 and 2021")
    return(YEAR)

In [7]:
def storm_Ident(NAME, YEAR):
    hurdat3 = hurdat2.loc[(hurdat4['name']==NAME) & (hurdat4['date'].str[:4]==YEAR)]
    return(hurdat3.storm_id.unique())


In [8]:
def lat_lon_to_float(v):
    #Convert strings from NHC to float locations
    
    if (v[-1] == 'S') or (v[-1] == 'W'):
        multiplier = -1
    else:
        multiplier = 1
    return(float(v[:-1]) * multiplier)

In [9]:
def get_distance(pair):
    #function to find the distance a storm has travelled between observations
    R = 6370
    point1 = pair[0]
    point2 = pair[1]
    lat1 = math.radians(point1[0])  #insert value
    lon1 = math.radians(point1[1])
    lat2 = math.radians(point2[0])
    lon2 = math.radians(point2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c
    return(distance)

In [10]:
def pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

In [11]:
data = []

with open('hurdat2-1851-2021-041922.txt', 'r') as f:
    for line in f.readlines():
        if line.startswith('AL'):
            #create a list of the items sep ','
            storm_id = line.split(',')
            #select first item and strip white space
            storm_number = storm_id[0].strip()
            storm_name = storm_id[1].strip()
        else:
            location_line = line.split(',')
            #strp is 'string parse'
            dt = datetime.strptime(location_line[0] + location_line[1], '%Y%m%d %H%M')
            storm_status = location_line[3].strip()
            storm_lat = lat_lon_to_float(location_line[4].strip())
            storm_lon = lat_lon_to_float(location_line[5].strip())
            max_sust = location_line[6].strip()
            min_pres = location_line[7].strip()
            data.append([dt, storm_number, storm_name, storm_status, storm_lat, storm_lon, max_sust, min_pres])
        

In [12]:
hurdat2 = pd.DataFrame(data, columns=['date', 'storm_id', 'name', 'cat', 'lat', 'long', 'max_sust_windspeed(knots)', 'min_pressure(millibars)'])


In [13]:
def uaf_df(first1, last1):
    #This function draws down data from the magnetometers located in Alaska
    start = "https://www.gi.alaska.edu/api/magnetometer/DATA/www/"
    bs = "/"
    us = "_"
    end = ".csv.tar.gz"
    stations = ['eagle', 'fortyukon', 'gakona', 'kenai', 'poker']
    yy = first1[:4]
    mm = first1[5:7]
    dd = first1[8:10]
    y1 = last1[:4]
    m1 = last1[5:7]
    d1 = last1[8:10]
    df = pd.date_range(start= yy + mm + dd, end= y1 + m1 + d1, freq='D')
    rng = df.format(formatter=lambda x: x.strftime('%Y/%m/%d'))
    rng2 = df.format(formatter=lambda x: x.strftime('%Y_%m_%d'))
    rng3 = list(zip(rng, rng2))
    frames=[]
    for i in rng3:
        for c in stations:
            try:
                url = start + i[0] + bs + c + bs + c + us + i[1] + end
                csv = urlopen(url)
                df1 = pd.read_csv(csv, compression='gzip', header=None, sep=',', on_bad_lines='skip')
                frames.append(df1)
            except:
                continue
    result = pd.concat(frames)
    result = result.dropna()
    return(result)
    


In [56]:
def process():
    data2 = []

    with open('geomag2.txt', 'r') as f:
        for line in f.readlines():
            if line.startswith('ea') or line.startswith('fo') or line.startswith('ga') or line.startswith('ke') or line.startswith('po'):
                #create a list of the items sep ','
                mags = line.split(',')
                station_id = mags[0].split('_')
                #select first item and strip white space
                station = station_id[0].strip()
                dt = datetime.strptime(station_id[1].strip() + station_id[2].strip() + station_id[3].strip()[:2] + '000000', '%Y%m%d%H%M%S')
                dec_time = float(mags[1].strip())
                H = float(mags[2].strip())
                D = float(mags[3].strip())
                Z = float(mags[4].strip())
                data2.append([station, dt, dec_time, H, D, Z])
            else:
                location_line = line.split(',')
                time = location_line[0].split(':')
                #strp is 'string parse'
                try:
                    dt1 = datetime.strptime(station_id[1].strip() + station_id[2].strip() + station_id[3].strip()[:2] + time[0].strip() + time[1].strip() + time[2].strip(), '%Y%m%d%H%M%S')
                except:
                    dt1 = datetime.strptime(station_id[1].strip() + station_id[2].strip() + station_id[3].strip()[:2] + time[0].strip() + time[1].strip() + '00', '%Y%m%d%H%M%S')
                dec_time = float(location_line[1].strip())
                H = float(location_line[2].strip())
                D = float(location_line[3].strip())
                Z = float(location_line[4].strip())
                data2.append([station, dt1, dec_time, H, D, Z])            
    df2 = pd.DataFrame(data2, columns = ['station', 'date', 'dec_time', 'H', 'D', 'Z'])
    return(df2)

In [57]:
def process_2(df2):
    cols = ['station', 'hour']
    options = ['00:00:00', '06:00:00', '12:00:00', '18:00:00']
    #get total magnetic field
    df2['M'] = np.sqrt((df2.H**2)+(df2.D**2)+(df2.Z**2))
    #get only sydonical time (every 6 hours)
    df2['hour']=df2['date'].astype(str)    
    df2['hour'] = df2.hour.str[-8:]    
    # selecting rows based on condition
    df3 = df2.loc[df2['hour'].isin(options)]
    df3 = df3.drop_duplicates()
    df4 = df3.drop(cols, axis=1)
    #get the mean values of all five stations
    df5 = df4.groupby('date').mean()
    df5 = df5.reset_index()
    return(df5)

In [58]:
def display():
    print('Enter name of Hurricane or \'Exit\' to quit.')

In [79]:
start = "http://geomag.usgs.gov/ws/data/?id="
typed = "&type=quasi-definitive&starttime="
end = "Z&endtime="
Z = "Z"

p, dp='AL062001', 'AL052019'

cols=['dates', 'time', 'DOY', 'H', 'D','Z','M']
to_drop = ['dates', 'time']

stations = ['BSL', 'FRN', 'SJG']
stat = 'SJG'

stillRunning = True

while(stillRunning):
    display()
    msg="Input Hurricane ID: "
    userSelection = (input(msg))
    userSelection = userSelection.upper()
    if userSelection[:2]=="AL" and userSelection[3:].isdigit() and len(userSelection)==8:
            storm_id = hurdat2.loc[hurdat2['storm_id']==userSelection]
            first, last = storm_id.date[~storm_id.date.isna()].values[[0]], storm_id.date[~storm_id.date.isna()].values[[-1]]
            first1, last1 = str(first[0])[:19], str(last[0])[:19]
            
            url = start + stat + typed + first1 + end + last1 + Z
            csv = urlopen(url)
            df = pd.read_csv(csv, sep=',', on_bad_lines='skip')
            df.to_csv('usgs' + stat + last1 + '.csv', index=False)
            df = pd.read_fwf('usgs' + stat + last1 + '.csv', skiprows=18)
            df.columns = cols
            df['date'] = df['dates'] + ' ' + df.time.str[:8]

            df.drop(to_drop, axis=1, inplace=True)
            check = len(df.M.unique())
            if check == 1:
                result = uaf_df(first1, last1)
                result.to_csv('geomag2.txt', sep=',', header=None, index=False)
                df9 = process_2(process())
            else:
                df8 = df[(df.M < 99999.0)]
                df9 = df8.drop_duplicates()
                df9.date = df9.date.astype('datetime64')
                storm_id.date = storm_id.date.astype('datetime64')
            df2 = pd.merge(storm_id, df9, on='date')
            df2.reset_index(drop=True, inplace=True)

            k = list((pairwise(list(zip(df2.lat, df2.long)))))
            sk = [0]
            for i in k:
                sk.append(get_distance(i))
            df2['dist'] = sk
            #forward motion
            df2['FM'] = df2.dist/6
            # Initialise the objects using StandardScaler() 
            sc_m = preproc.minmax_scale(df2["M"])
            sc_y = preproc.minmax_scale(df2["FM"])
            sc_h = preproc.minmax_scale(df2["H"])
            sc_d = preproc.minmax_scale(df2["D"])
            sc_z = preproc.minmax_scale(df2["Z"])
            fig = go.Figure()

            # Add traces
            fig.add_trace(go.Scatter(x=df2["date"], y=sc_m,
                                mode='lines',
                                name='M',
                                line=dict(width=0.75)))
            fig.add_trace(go.Scatter(x=df2["date"], y=sc_h,
                                mode='lines',
                                name='H',
                                line=dict(width=0.75)))
            fig.add_trace(go.Scatter(x=df2["date"], y=sc_d,
                                name='D',
                                line=dict(width=0.75)))
            fig.add_trace(go.Scatter(x=df2["date"], y=sc_z,
                                mode='lines',
                                name='Z',
                                line=dict(width=0.75)))
            fig.add_trace(go.Scatter(x=df2["date"], y=sc_y,
                                mode='lines',
                                name='FM',
                                line=dict(width=0.75)))

            fig.show()
    elif userSelection == 'EXIT':
        print("Bye Bye")
        stillRunning = False      
    else:
        print("It appears this database has no record for: " + userSelection + '\nAre you sure you have the correct input value?')
        




Enter name of Hurricane or 'Exit' to quit.
Input Hurricane ID: AL052019


Enter name of Hurricane or 'Exit' to quit.
Input Hurricane ID: Exit
Bye Bye


In [78]:
#Applet for getting the Storm from Name and Year
msg="Input Hurricane Name: "
NAME = input(msg)
NAME = NAME.upper()
YEAR = integerValidation()
userselection = storm_Ident(NAME, str(YEAR))[0]
print(userselection)

Input Hurricane Name: Dorian
Input Year: 2019
AL052019


In [80]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

import warnings # supress warnings
warnings.filterwarnings('ignore')

In [81]:
df2.head()

,date,storm_id,name,cat,lat,long,max_sust_windspeed(knots),min_pressure(millibars),DOY,H,D,Z,M,dist,FM
0,2019-08-24 06:00:00,AL052019,DORIAN,TD,10.3,-46.4,25,1011,236,26371.6,-6107.9,24870.1,36759.49,0.000000,0.000000
1,2019-08-24 12:00:00,AL052019,DORIAN,TD,10.4,-47.5,30,1010,236,26375.3,-6089.5,24861.2,36753.07,120.817852,20.136309
2,2019-08-24 18:00:00,AL052019,DORIAN,TS,10.6,-48.7,35,1008,236,26384.6,-6131.5,24859.3,36765.54,133.049973,22.174996
3,2019-08-25 00:00:00,AL052019,DORIAN,TS,10.8,-49.9,35,1008,237,26367.3,-6110.5,24866.7,36754.64,132.965510,22.160918
4,2019-08-25 06:00:00,AL052019,DORIAN,TS,11.0,-51.0,35,1008,237,26364.3,-6102.6,24868.9,36752.46,122.129926,20.354988


In [82]:
# filter only Production and Export Quantity
df6 = df2.loc[:, ['date', 'FM', 'M', 'H']]
df6.head()

,date,FM,M,H
0,2019-08-24 06:00:00,0.000000,36759.49,26371.6
1,2019-08-24 12:00:00,20.136309,36753.07,26375.3
2,2019-08-24 18:00:00,22.174996,36765.54,26384.6
3,2019-08-25 00:00:00,22.160918,36754.64,26367.3
4,2019-08-25 06:00:00,20.354988,36752.46,26364.3


In [83]:
# Declare a variable named as 'X' and 'y'
X = df6.drop('FM', axis=1)             # All features (independent varaibles) except MEDV 
y = df6['FM'].values                    # Target variable

# Split the data into 70% and 30% by using a parameter test_size = 30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

# Display the size of the rows and columns
X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape

((69, 3), (69,), (51, 3), (18, 3), (51,), (18,))

In [84]:
# first model with an arbitrary choice of n_features
# running RFE with number of features=3

lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select=2)             
rfe = rfe.fit(X_train, y_train)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>)

In [72]:
# tuples of (feature name, whether selected, ranking)
# note that the 'rank' is > 1 for non-selected features
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

[('M', True, 1), ('H', True, 1)]

In [73]:
# predict Export Quantities of X_test
y_pred = rfe.predict(X_test)

# evaluate the model on test set
r2 = sklearn.metrics.r2_score(y_test, y_pred)
print(r2)

0.08935201752769328


In [ ]:
#The R-squared values here represent the amount of causality
#between the dependent and independent variables
#it is clear that there is very little causation in any of these examples
#but there is slightly more causality present in the hurricanes which
#are suspected of having been tampered with
"Irene: 2005 = -2.0129363242378635"
"Irene: 2011 = 0.21632780597213241"
"Dorian 2019 = 0.05929296441404075"
"OPHELIA 2005 -0.2958445224085673"
"ERIN 2001 0.08935201752769328"

In [ ]:
#plot all hurricanes (need to pivot hurdat for this)
#install pyspark etc.
#address extraneous values the sun
#make a notebook on the 

In [74]:
print(NAME, YEAR, r2)

ERIN 2001 0.08935201752769328


In [86]:
df4 = df6.set_index('date')
df4.head()

,FM,M,H
date,,,
2019-08-24 06:00:00,0.000000,36759.49,26371.6
2019-08-24 12:00:00,20.136309,36753.07,26375.3
2019-08-24 18:00:00,22.174996,36765.54,26384.6
2019-08-25 00:00:00,22.160918,36754.64,26367.3
2019-08-25 06:00:00,20.354988,36752.46,26364.3


In [90]:
#this failed and was probably not important anyway
#attempting to predict the trend of the FM based on the trend of 
#M was likely never going to work and would lack detail

from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
%matplotlib inline

# Multiplicative Decomposition 
result_mul = seasonal_decompose(df4['FM'], model='multiplicative', extrapolate_trend='freq')

# Additive Decomposition
result_add = seasonal_decompose(df4['FM'], model='additive', extrapolate_trend='freq')

# Plot
plt.rcParams.update({'figure.figsize': (10,10)})
result_mul.plot().suptitle('Multiplicative Decompose', fontsize=22)
result_add.plot().suptitle('Additive Decompose', fontsize=22)
plt.show()


ValueError: You must specify a period or x must be a pandas object with a PeriodIndex or a DatetimeIndex with a freq not set to None

In [ ]:
#Look for trend and decompose data

In [104]:
%matplotlib inline

df7 = df6.dropna()

df7['date'] = pd.to_datetime(df7['date'])
df7 = df7.set_index('date').asfreq('6H')
df7.head()

,FM,M,H
date,,,
2019-08-24 06:00:00,0.000000,36759.49,26371.6
2019-08-24 12:00:00,20.136309,36753.07,26375.3
2019-08-24 18:00:00,22.174996,36765.54,26384.6
2019-08-25 00:00:00,22.160918,36754.64,26367.3
2019-08-25 06:00:00,20.354988,36752.46,26364.3


In [109]:
nulls = df7.M.isnull().values.any()
print(nulls)

True


In [111]:
#There are missing values between 48 and 50
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(y=df7["FM"],
                    mode='lines',
                    name='FM',
                    line=dict(width=0.75)))

fig.update_layout(
    title_text='Forward Motion of Hurricane Dorian (2019)', # title of plot
    xaxis_title_text='Time (6 hr increments)', # xaxis label
    yaxis_title_text='kph', # yaxis label
)

fig.show()

In [113]:
#There are missing values between 48 and 50
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(y=df7["M"],
                    mode='lines',
                    name='M',
                    line=dict(width=0.75)))
fig.update_layout(
    title_text='Total Intensity Geomagnetic Field', # title of plot
    xaxis_title_text='Time (6 hr increments)', # xaxis label
    yaxis_title_text='B(nT)', # yaxis label
)

fig.show()

In [118]:
#use interpolate to fill the gaps
df8 = df7.interpolate(method='linear')
#There are missing values between 48 and 50
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(y=df8["M"],
                    mode='lines',
                    name='M',
                    line=dict(width=0.75)))

fig.update_layout(
    title_text='Total Intensity Geomagnetic Field', # title of plot
    xaxis_title_text='Time (6 hr increments)', # xaxis label
    yaxis_title_text='B(nT)', # yaxis label
)

fig.show()

In [115]:
nulls = df8.M.isnull().values.any()
print(nulls)

False


In [119]:
df8.reset_index(inplace=True)
df01 = df8.set_index('date').asfreq('6H')


TypeError: Index(...) must be called with a collection of some kind, 'seasonal' was passed

In [120]:
df8.head()

,date,FM,M,H
0,2019-08-24 06:00:00,0.000000,36759.49,26371.6
1,2019-08-24 12:00:00,20.136309,36753.07,26375.3
2,2019-08-24 18:00:00,22.174996,36765.54,26384.6
3,2019-08-25 00:00:00,22.160918,36754.64,26367.3
4,2019-08-25 06:00:00,20.354988,36752.46,26364.3


In [125]:
df01.head()

,FM,M,H
date,,,
2019-08-24 06:00:00,0.000000,36759.49,26371.6
2019-08-24 12:00:00,20.136309,36753.07,26375.3
2019-08-24 18:00:00,22.174996,36765.54,26384.6
2019-08-25 00:00:00,22.160918,36754.64,26367.3
2019-08-25 06:00:00,20.354988,36752.46,26364.3


In [126]:
df8 = df8.set_index('date').asfreq('6H')
result_add = seasonal_decompose(df8['FM'], model='additive', extrapolate_trend='freq')

result = seasonal_decompose(df01, model='ad')

from pylab import rcParams
rcParams['figure.figsize'] = 12,5
result.plot();

TypeError: Index(...) must be called with a collection of some kind, 'seasonal' was passed

In [92]:
plt.rcParams.update({'figure.figsize': (10,10)})
result_mul.plot().suptitle('Multiplicative Decompose', fontsize=22)
result.plot().suptitle('Additive Decompose', fontsize=22)
plt.show()

NameError: name 'plt' is not defined